In [30]:
import formats.futures_strategy_formats as fsf
import formats.strategy_followup_formats as sff
import formats.risk_pnl_formats as rpf
import my_sql_routines.futures_price_loader as fpl
import my_sql_routines.my_sql_utilities as msu
import get_price.presave_price as pp
import opportunity_constructs.futures_butterfly as fb
import opportunity_constructs.overnight_calendar_spreads as ocs
import contract_utilities.expiration as exp
import ta.prepare_daily as prep
import ta.underlying_proxy as up
import fundamental_data.cot_data as cot
import datetime as dt
import ta.email_reports as er

con = msu.get_my_sql_connection()

report_date = exp.doubledate_shift_bus_days()


try:
    writer_out = sff.generate_futures_butterfly_followup_report(as_of_date=report_date, con=con)
    writer_out = sff.generate_spread_carry_followup_report(as_of_date=report_date, con=con, writer=writer_out)
    writer_out = sff.generate_vcs_followup_report(as_of_date=report_date, con=con, writer=writer_out)
    sff.generate_ocs_followup_report(as_of_date=report_date, con=con, broker='abn',writer=writer_out)
    prep.move_from_dated_folder_2daily_folder(ext='ta', file_name='followup', folder_date=report_date)
except Exception:
    pass

try:
    er.send_hrsn_report(report_date=report_date, con=con)
except Exception:
    pass

In [29]:
import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore",category=FutureWarning)
    import h5py

import shared.log as lg
log = lg.get_logger(file_identifier='evening_job',log_level='INFO')

import shared.directory_names as dn
import shared.downloads as sd
import shared.calendar_utilities as cu
import pickle as pickle
import my_sql_routines.futures_price_loader as fpl
import my_sql_routines.options_price_loader as opl
import my_sql_routines.options_greek_loader as ogl
import my_sql_routines.options_signal_loader as osl
import my_sql_routines.my_sql_utilities as msu
import get_price.presave_price as pp
import opportunity_constructs.vcs as vcs
import formats.options_strategy_formats as osf
import formats.futures_strategy_formats as fsf
import ta.prepare_daily as prep

folder_date = 20181115

con = msu.get_my_sql_connection()

try:
    log.info('generate_vcs_sheet')
    vcs.generate_vcs_sheet_4date(con=con,date_to=folder_date)
except Exception:
    log.error('generate_vcs_sheet failed', exc_info=True)
    pass

try:
    osf.generate_vcs_formatted_output(report_date=folder_date)
    prep.prepare_strategy_daily(strategy_class='vcs', report_date=folder_date)
except Exception:
    pass

try:
    log.info('generate_scv_sheet')
    osf.generate_scv_formatted_output(report_date=folder_date)
    prep.prepare_strategy_daily(strategy_class='scv', report_date=folder_date)
except Exception:
    log.error('generate_scv_sheet failed', exc_info=True)
    pass

try:
    log.info('generate_ifs')
    fsf.generate_ifs_formatted_output(report_date=folder_date)
except Exception:
    log.error('generate_ifs failed', exc_info=True)
    pass

con.close()

C:\Research\PycharmProjects\opportunity_constructs\intraday_future_spreads.py:79: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  merged112,merged122,merged110,merged100])


In [1]:
import my_sql_routines.options_signal_loader as osl
import shared.calendar_utilities as cu
import my_sql_routines.my_sql_utilities as msu

con = msu.get_my_sql_connection()
folder_date = 20181115
osl.load_ticker_signals_4settle_date(con=con, settle_date=folder_date)

In [28]:
atm_vol_frame.iloc[380:390]

,id,ticker,expiration_date,cal_dte,delta_vol,strike,theta,tr_dte,open_interest,volume,real_vol,ticker_head,ticker_month,ticker_year
380,48140,CLX2020,2020-10-15,700.0,NaN,NaN,NaN,482.0,40.0,0.0,28.243702,CL,11,2020
381,48258,CLZ2019,2019-11-15,365.0,0.331927,60.50,-0.009697,252.0,461329.0,19912.0,31.065732,CL,12,2019
382,48259,CLZ2020,2020-11-17,733.0,0.277292,60.50,-0.005152,505.0,108066.0,505.0,28.154301,CL,12,2020
383,48377,NGF2019,2018-12-26,41.0,0.905063,4.25,-0.005919,27.0,45037.0,12081.0,102.432328,NG,1,2019
384,48378,NGF2020,2019-12-26,406.0,0.240347,2.95,-0.000329,279.0,8.0,0.0,8.810030,NG,1,2020
385,48496,NGG2019,2019-01-28,74.0,0.855473,4.20,-0.003988,48.0,58349.0,5623.0,105.306002,NG,2,2019
386,48497,NGG2020,2020-01-28,439.0,0.263449,3.00,-0.000344,300.0,6.0,0.0,8.086146,NG,2,2020
387,48615,NGH2019,2019-02-25,102.0,0.830535,3.95,-0.003027,67.0,53918.0,1630.0,113.773267,NG,3,2019
388,48616,NGH2020,2020-02-25,467.0,0.284180,3.00,-0.000332,319.0,6.0,0.0,7.744173,NG,3,2020
389,48734,NGJ2019,2019-03-26,131.0,0.285178,2.85,-0.000707,88.0,8684.0,1077.0,18.280003,NG,4,2019


In [27]:
import datetime as dt
import shared.calendar_utilities as cu
import contract_utilities.contract_meta_info as cmi
import numpy as np

now = dt.datetime.now()
folder_date = 20181115
settle_datetime = cu.convert_doubledate_2datetime(folder_date)

contract_specs_output = [cmi.get_contract_specs(x) for x in atm_vol_frame['ticker']]

atm_vol_frame['ticker_head'] = [x['ticker_head'] for x in contract_specs_output]
atm_vol_frame['ticker_month'] = [x['ticker_month_num'] for x in contract_specs_output]
atm_vol_frame['ticker_year'] = [x['ticker_year'] for x in contract_specs_output]

column_names = atm_vol_frame.columns.tolist()

ticker_indx = column_names.index('ticker')
cal_dte_indx = column_names.index('cal_dte')
tr_dte_indx = column_names.index('tr_dte')
ticker_head_indx = column_names.index('ticker_head')
ticker_month_indx = column_names.index('ticker_month')
ticker_year_indx = column_names.index('ticker_year')
imp_vol_indx = column_names.index('delta_vol')
strike_indx = column_names.index('strike')
theta_indx = column_names.index('theta')
open_interest_indx = column_names.index('open_interest')
volume_indx = column_names.index('volume')
real_vol_indx = column_names.index('real_vol')

atm_vol_frame_select = atm_vol_frame.iloc[390:400]


tuples = [tuple([x[ticker_indx], x[ticker_head_indx], x[ticker_month_indx] , x[ticker_year_indx],
                     None if np.isnan(x[cal_dte_indx]) else x[cal_dte_indx],
                     None if np.isnan(x[tr_dte_indx]) else x[tr_dte_indx],
                     None if np.isnan(x[imp_vol_indx]) else 100*x[imp_vol_indx], 0.5,
                     None if np.isnan(x[theta_indx]) else x[theta_indx],
                     None if np.isnan(x[strike_indx]) else x[strike_indx],
                     None if np.isnan(x[real_vol_indx]) else x[real_vol_indx],
                     None if np.isnan(x[open_interest_indx]) else x[open_interest_indx],
                     None if np.isnan(x[volume_indx]) else x[volume_indx],
                     settle_datetime.date(), now, now]) for x in atm_vol_frame_select.values]

column_str = "ticker, ticker_head, ticker_month, ticker_year, " \
             " cal_dte, tr_dte, imp_vol, delta, theta, strike, close2close_vol20, open_interest, volume, price_date, created_date, last_updated_date"

insert_str = ("%s, " * len(column_str.split(',')))[:-2]
final_str = "REPLACE INTO option_ticker_indicators (%s) VALUES (%s)" % (column_str, insert_str)

msu.sql_execute_many_wrapper(final_str=final_str, tuples=tuples, con=con)

In [1]:
import formats.futures_strategy_formats as fsf
fsf.generate_spread_carry_formatted_output()

In [39]:
import opportunity_constructs.spread_carry as sc

#Carry Spread Bet Size: 4291
spread_carry_output = sc.generate_spread_carry_sheet_4date(report_date=20181114)
spread_report = spread_carry_output['spread_report']
ticker_head_list = list(spread_report['tickerHead'].unique())
# Compare BO and S sizing



spread_report = spread_report[spread_report['tickerHead']==ticker_head_list[19]]
spread_report = spread_report[['ticker1','ticker2','tickerHead','front_tr_dte','carry','q_carry','reward_risk','price','q','upside','downside']]
spread_report
#spread_report[spread_report['tickerHead']=='FC']

# HO ZF might be a buy after the butterfly
# CT HK settle was 0.07

,ticker1,ticker2,tickerHead,front_tr_dte,carry,q_carry,reward_risk,price,q,upside,downside
0,EDZ2018,EDF2019,ED,22.0,NaN,NaN,NaN,0.040,76,31.25,-34.38
1,EDF2019,EDG2019,ED,40.0,0.0,0.0,0.00,0.040,76,43.75,-31.25
2,EDG2019,EDH2019,ED,65.0,-37.5,-12.0,-0.20,0.055,88,37.50,-42.50
3,EDH2019,EDJ2019,ED,85.0,62.5,26.0,0.31,0.030,62,50.00,-50.00
4,EDJ2019,EDM2019,ED,105.0,-175.0,-19.0,-0.54,0.100,81,81.25,-56.25
5,EDM2019,EDU2019,ED,148.0,50.0,33.0,0.09,0.080,48,75.75,-62.50
6,EDU2019,EDZ2019,ED,211.0,37.5,11.0,0.06,0.065,37,75.00,-50.00
7,EDZ2019,EDH2020,ED,275.0,137.5,28.0,0.19,0.010,9,62.50,-56.25
8,EDH2020,EDM2020,ED,338.0,37.5,4.0,0.06,-0.005,5,50.00,-50.00
9,EDM2020,EDU2020,ED,401.0,12.5,3.0,0.03,-0.010,2,44.50,-37.50


In [25]:
import signals.overnight_calendar_spread_signals as ocss
import numpy as np
signals_output = ocss.get_overnight_calendar_signals(ticker_list=['CLM2017','CLN2017'],date_to=20170104)

signals_output


KeyError: 'butterfly_z'

In [3]:
import opportunity_constructs.overnight_calendar_spreads as ocs
wuhu = ocs.generate_overnight_spreads_sheet_4date(date_to=20170106)
wuhu['overnight_calendars']

,ticker1,ticker2,tickerHead,tickerClass,trDte1,trDte2,multiplier,agg,cBack,ts_slope5,ts_slope10,linear_deviation5,linear_deviation10,momentum5,momentum10,underlying_zscore,spread_price,normalized_target,noise_100,contract_multiplier
0,LNG2017,LNJ2017,LN,Livestock,26,68,400,12,20,-9.121593e-01,-0.366469,-4.402966e-02,-8.115648e-01,-5.198327e+00,-2.206837e+00,-1.304554,-4.225,NaN,0.509779,400
1,LNJ2017,LNM2017,LN,Livestock,68,109,400,12,20,-1.330140e-01,0.165085,-5.748711e-01,-1.046335e+00,7.518181e-01,-3.469930e-01,-0.986898,-9.000,NaN,0.432285,400
2,LNM2017,LNN2017,LN,Livestock,109,131,400,12,20,-3.521164e-01,0.076025,7.387203e-01,1.442144e-01,1.430473e+00,-2.200727e-01,-0.824638,0.375,NaN,0.227198,400
3,LNN2017,LNQ2017,LN,Livestock,131,151,400,12,20,9.079230e-01,0.422661,7.962175e-01,1.264728e+00,4.473824e+00,5.526488e+00,1.249454,0.900,NaN,0.189994,400
4,LNQ2017,LNV2017,LN,Livestock,151,194,400,12,20,6.663268e-01,0.029370,9.908674e-01,2.102287e+00,2.826841e+00,-1.009586e-01,2.071181,10.750,NaN,0.247626,400
5,LCG2017,LCJ2017,LC,Livestock,35,77,400,12,20,-4.561672e-01,-0.490725,-8.708771e-02,-2.874893e-01,-4.226255e+00,-3.085837e+00,-0.794333,0.625,NaN,0.372670,400
6,LCJ2017,LCM2017,LC,Livestock,77,121,400,12,20,3.070317e-01,0.310133,-4.241466e-01,-5.130627e-01,-5.117196e-01,1.242748e+00,1.078998,10.225,NaN,0.341984,400
7,LCM2017,LCQ2017,LC,Livestock,121,164,400,12,20,-2.159053e-01,-0.370965,-2.934257e-01,-6.399672e-03,-2.806769e+00,-2.482911e+00,-0.393368,3.750,NaN,0.231583,400
8,LCQ2017,LCV2017,LC,Livestock,164,206,400,12,20,-2.952664e-01,-0.224158,-6.033084e-01,-1.022533e+00,-2.175647e+00,-1.554034e+00,-1.514845,0.125,NaN,0.160872,400
9,LCV2017,LCZ2017,LC,Livestock,206,247,400,12,20,-4.521950e-01,-0.085154,-7.869910e-01,-1.360855e+00,-1.614982e+00,-2.583971e+00,-1.808900,-0.450,NaN,0.154800,400


In [12]:

import shared.statistics as stats
stats.get_regression_results({'y':selected_data['spread_price'],'x':selected_data['c1']['close_price']})


{'alpha': -0.067272386661693773,
 'beta': -0.0038169726489317712,
 'conf_int': array([[-4.05040882,  3.91586405],
        [-0.07461787,  0.06698392]]),
 'residualstd': 0.049146045119995069,
 'rsquared': 0.19282020922796184,
 'zscore': -0.92821729511873996}

In [13]:
last_10_data = aligned_data.iloc[-10:]
last_5_data = aligned_data.iloc[-5:]
last_5_data

c1                                                      \
              ticker ticker_head ticker_month ticker_year cal_dte tr_dte   
settle_date                                                                
2016-12-30   CLM2017          CL            6        2017     143     97   
2017-01-03   CLM2017          CL            6        2017     139     96   
2017-01-04   CLM2017          CL            6        2017     138     95   
2017-01-05   CLM2017          CL            6        2017     137     94   
2017-01-06   CLM2017          CL            6        2017     136     93   

                                                            ...           c2  \
            open_price high_price low_price close_price     ...      change2   
settle_date                                                 ...                
2016-12-30       56.68      56.84     56.24       56.49     ...         1.36   
2017-01-03       56.83      57.95     55.13       55.32     ...         0.58   
2017-01-04       55.58      56.44     55.17       56.29     ...          NaN   
2017-01-05       56.38      57.03     55.74       56.68     ...          NaN   
2017-01-06       56.61      57.13     56.14       56.88     ...          NaN   

                                                        settle_date  \
            change5 change10 change20 change_5 change_1               
settle_date                                                           
2016-12-30      NaN      NaN      NaN     0.92    -0.06  2016-12-30   
2017-01-03      NaN      NaN      NaN    -0.27    -1.11  2017-01-03   
2017-01-04      NaN      NaN      NaN    -0.03     0.98  2017-01-04   
2017-01-05      NaN      NaN      NaN     0.13     0.38  2017-01-05   
2017-01-06      NaN      NaN      NaN     0.39     0.20  2017-01-06   

            tr_dte_match spread_change_1 spread_price  
                                                       
settle_date                                            
2016-12-30             4    7.105427e-15        -0.27  
2017-01-03             3   -6.000000e-02        -0.33  
2017-01-04             2   -1.000000e-02        -0.34  
2017-01-05             1    1.000000e-02        -0.33  
2017-01-06             0    0.000000e+00        -0.33  

[5 rows x 44 columns]

In [7]:
import signals.technical_indicators as ti
ti.time_series_slope(data_frame_input=aligned_data,num_obs=5,y_var_name='spread_price')

-0.012000000000000469

In [7]:


import contract_utilities.contract_meta_info as cmi
import opportunity_constructs.utilities as opUtil
import get_price.get_futures_price as gfp

ticker1 = 'CLG2017'
ticker2 = 'CLH2017'
aggregation_method = 3
contracts_back = 8
date_to = 20170105

ticker_head = cmi.get_contract_specs(ticker1)

use_last_as_current = False

aligned_output = opUtil.get_aligned_futures_data(contract_list=[ticker1, ticker2],
                                                          aggregation_method=aggregation_method,
                                                          contracts_back=contracts_back,
                                                          date_to=date_to,
                                                          use_last_as_current=use_last_as_current)

aligned_data = aligned_output['aligned_data']

In [12]:
selected_data = aligned_data[aligned_data['c1']['cont_indx']==201702]
selected_data['spread'] = selected_data['c1']['close_price']-selected_data['c2']['close_price']

In [10]:

import matplotlib.pyplot as plt

plt.scatter(selected_data['c1']['close_price'],selected_data['spread_price'])
plt.grid()         
         
plt.show()
